# BasicMotions Classification with Granite Time Series TSPulse

TSPulse are compact pre-trained models specialized for various multivariate time-series analysis tasks namely classification, anomaly-detection, imputation and similarity search, open-sourced by IBM Research. With 1 Million parameters, TSPulse introduces the notion of the first-ever "tiny" pre-trained models for Time-Series tasks like classification, anomaly-detection, imputation and similarity search. TSPulse outperforms several popular benchmarks demanding billions of parameters in zero-shot and few-shot setting and can easily be fine-tuned for different downstream tasks.

In this recipe, we demonstrates the usage of a pre-trained TSPulse model for time-series classification task. This example makes use of the [BasicMotions](https://www.timeseriesclassification.com/description.php?Dataset=BasicMotions) dataset from the [UEA](https://arxiv.org/abs/1811.00075) multivariate time-series classification archive.

The data was generated as part of a student project where four students performed four activities whilst wearing a smart watch. The watch collects 3D accelerometer and a 3D gyroscope. The data order is accelerometer x, y, z then gyroscope x, y, z. There are four different classes: walking, resting, running and badminton.


## Install the TSFM Library

The [granite-tsfm library](https://github.com/ibm-granite/granite-tsfm) provides utilities for working with Time Series Foundation Models (TSFM). Here the pinned version is retrieved and installed.


In [ ]:
# Install the tsfm library
! pip install "granite-tsfm[notebooks]==0.2.28"

## Imports

In [ ]:
import math
import os
import tempfile
import requests
import zipfile
import io
import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, random_split
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.data.data_collator import default_data_collator
from transformers.trainer_utils import RemoveColumnsCollator

### Imports from `tsfm_public`

In [ ]:
from tsfm_public.models.tspulse import TSPulseForClassification
from tsfm_public.toolkit.dataset import ClassificationDFDataset
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.time_series_classification_preprocessor import TimeSeriesClassificationPreprocessor
from tsfm_public.toolkit.util import convert_tsfile_to_dataframe

In [ ]:
# set the seed
seed = 42
set_seed(seed)

## Data Preprocessing

We will download the BasicMotions dataset and extract the train and test `.ts files`.

In [ ]:
url = "https://www.timeseriesclassification.com/aeon-toolkit/BasicMotions.zip"

extract_dir = "BasicMotions"
os.makedirs(extract_dir, exist_ok=True)

print("Downloading ZIP file...")
response = requests.get(url)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    ts_files = [f for f in zip_ref.namelist() if f.endswith(".ts")]

    print(f"Extracting {len(ts_files)} .ts files...")
    for file in ts_files:
        zip_ref.extract(file, extract_dir)

print(f"Extraction completed. Files are saved in '{extract_dir}'")

## Read in the data and train the preprocessor

In [ ]:
dataset_name = "BasicMotions"

In [ ]:
path = f"{dataset_name}/{dataset_name}_TRAIN.ts"

df_base = convert_tsfile_to_dataframe(
    path,
    return_separate_X_and_y=False,
)
label_column = "class_vals"
input_columns = [f"dim_{i}" for i in range(df_base.shape[1] - 1)]

tsp = TimeSeriesClassificationPreprocessor(
    input_columns=input_columns,
    label_column=label_column,
    scaling=True,
)

tsp.train(df_base)
df_prep = tsp.preprocess(df_base)
base_dataset = ClassificationDFDataset(
    df_prep,
    id_columns=[],
    timestamp_column=None,
    input_columns=input_columns,
    label_column=label_column,
    context_length=512,
    static_categorical_columns=[],
    stride=1,
    enable_padding=False,
    full_series=True,
)

path = f"{dataset_name}/{dataset_name}_TRAIN.ts"

df_test = convert_tsfile_to_dataframe(
    path,
    return_separate_X_and_y=False,
)
label_column = "class_vals"
input_columns = [f"dim_{i}" for i in range(df_test.shape[1] - 1)]

tsp = TimeSeriesClassificationPreprocessor(
    input_columns=input_columns,
    label_column=label_column,
    scaling=True,
)

tsp.train(df_test)
df_prep = tsp.preprocess(df_test)

test_dataset = ClassificationDFDataset(
    df_prep,
    id_columns=[],
    timestamp_column=None,
    input_columns=input_columns,
    label_column=label_column,
    context_length=512,
    static_categorical_columns=[],
    stride=1,
    enable_padding=False,
    full_series=True,
)

## Creating a validation set 

In [ ]:
dataset_size = len(base_dataset)
print(dataset_size)
split_valid_ratio = 0.1
val_size = int(split_valid_ratio * dataset_size)  # 10% valid split
train_size = dataset_size - val_size
train_dataset, valid_dataset = random_split(base_dataset, [train_size, val_size])

## Configs for the TSPulse model

### Hyperparameters to Optimize and suggested values :
head_reduce_d_model = 1, 2

decoder_mode = mix_channel, common_channel

head_gated_attention_activation = softmax, sigmoid

mask_ratio = 0, 0.3

channel_virtual_expand_scale = 1, 2

In [ ]:
config_dict = {
    "head_gated_attention_activation": "softmax",
    "channel_virtual_expand_scale": 2,
    "mask_ratio": 0.3,
    "head_reduce_d_model": 1,
    "disable_mask_in_classification_eval": True,
    "fft_time_consistent_masking": True,
    "decoder_mode": "mix_channel",
    "head_aggregation_dim": "patch",
    "head_aggregation": None,
    "loss": "cross_entropy",
    "ignore_mismatched_sizes": True,
}

config_dict["num_input_channels"] = tsp.num_input_channels
config_dict["num_targets"] = df_base["class_vals"].nunique()

## Getting the Pretrained TSPulse Model from HuggingFace with above configs

In [ ]:
model = TSPulseForClassification.from_pretrained(
    "ibm-granite/granite-timeseries-tspulse-r1", revision="tspulse-block-dualhead-512-p16-r1", **config_dict
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
print(device)
model = model.to(device).float()

## Backbone of the pre-trained model is freezed and the classifier head along with the input patch embedding layer is finetuned on the classification dataset.

In [ ]:
# Freezing Backbone except patch embedding layer....

for param in model.backbone.parameters():
    param.requires_grad = False

for param in model.backbone.time_encoding.parameters():
    param.requires_grad = True
for param in model.backbone.fft_encoding.parameters():
    param.requires_grad = True

## Finetuning the classifier head and patch embedding layer

In [ ]:
OUT_DIR = "tspulse_finetuned_models/"

In [ ]:
temp_dir = tempfile.mkdtemp()

suggested_lr = None

train_dict = {"per_device_train_batch_size": 32, "num_train_epochs": 200, "eval_accumulation_steps": None}

EPOCHS = train_dict["num_train_epochs"]
BATCH_SIZE = train_dict["per_device_train_batch_size"]
eval_accumulation_steps = train_dict["eval_accumulation_steps"]
NUM_WORKERS = 1
NUM_GPUS = 1

set_seed(42)
if suggested_lr is None:
    lr, model = optimal_lr_finder(
        model,
        train_dataset,
        batch_size=BATCH_SIZE,
    )
    suggested_lr = lr
print("Suggested LR : ", suggested_lr)
finetune_args = TrainingArguments(
    output_dir=temp_dir,
    overwrite_output_dir=True,
    learning_rate=suggested_lr,
    num_train_epochs=EPOCHS,
    do_eval=True,
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_accumulation_steps=eval_accumulation_steps,
    dataloader_num_workers=NUM_WORKERS,
    report_to="tensorboard",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=1,
    logging_dir=os.path.join(OUT_DIR, "output"),  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
)

# Create the early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=100,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.0001,  # Minimum improvement required to consider as improvement
)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=suggested_lr)
scheduler = OneCycleLR(
    optimizer,
    suggested_lr,
    epochs=EPOCHS,
    steps_per_epoch=math.ceil(len(train_dataset) / (BATCH_SIZE * NUM_GPUS)),
)

finetune_trainer = Trainer(
    model=model,
    args=finetune_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
    optimizers=(optimizer, scheduler),
)

# Fine tune
finetune_trainer.train()

## Evaluating the model

In [ ]:
predictions_dict = finetune_trainer.predict(test_dataset)
preds_np = predictions_dict.predictions[0]

remove_columns_collator = RemoveColumnsCollator(
    data_collator=default_data_collator,
    signature_columns=["target_values"],
    logger=None,
    description=None,
    model_name="temp",
)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=remove_columns_collator)
target_list = []
for batch in test_dataloader:
    batch_labels = batch["target_values"].numpy()
    target_list.append(batch_labels)
targets_np = np.concatenate(target_list, axis=0)
test_accuracy = np.mean(targets_np == np.argmax(preds_np, axis=1))
print("test_accuracy : ", test_accuracy)